**Twitter-Sentiment-Analysis**

https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/



In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import re # for regular expressions
import pandas as pd 
pd.set_option("display.max_colwidth", 200)
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk # for text manipulation
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [0]:
train  = pd.read_csv("drive/My Drive/datasets/Twitter-Sentiment-Analysis/train_E6oV3lV.csv")
test = pd.read_csv("drive/My Drive/datasets/Twitter-Sentiment-Analysis/test_tweets_anuFYb8.csv")

In [76]:
train.shape, test.shape

((31962, 3), (17197, 2))

In [77]:
train.head(3)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty


In [78]:
test.head(3)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedication #willpower to find #newmaterialsâ¦
1,31964,@user #white #supremacists want everyone to see the new â #birdsâ #movie â and hereâs why
2,31965,safe ways to heal your #acne!! #altwaystoheal #healthy #healing!!


In [79]:
train["label"].value_counts()

0    29720
1     2242
Name: label, dtype: int64

In [81]:
combi = train.append(test, ignore_index=True)
combi.shape

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


(49159, 3)

In [82]:
combi.tail(3)

,id,label,tweet
49156,49157,NaN,"#hillary #campaigned today in #ohio((omg)) &amp; used words like ""assets&amp;liability"" never once did #clinton say thee(word) #radicalization"
49157,49158,NaN,"happy, at work conference: right mindset leads to culture-of-development organizations #work #mindset"
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze #newmusic #newsong"


In [0]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [84]:
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*") 
combi.head()

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0.0,factsguide: society now #motivation,factsguide: society now #motivation


In [85]:
combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
combi.head(10)

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction #run
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for #lyft credit i can t use cause they don t offer wheelchair vans in pdx #disapointed #getthanked
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in ur
4,5,0.0,factsguide: society now #motivation,factsguide society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo,huge fan fare and big talking before they leave chaos and pay disputes when they get there #allshowandnogo
6,7,0.0,@user camping tomorrow @user @user @user @user @user @user @user dannyâ¦,camping tomorrow danny
7,8,0.0,the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl,the next school year is the year for exams can t think about that #school #exams #hate #imagine #actorslife #revolutionschool #girl
8,9,0.0,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦,we won love the land #allin #cavs #champions #cleveland #clevelandcavaliers
9,10,0.0,@user @user welcome here ! i'm it's so #gr8 !,welcome here i m it s so #gr


In [0]:
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [87]:
combi.head()

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when father dysfunctional selfish drags kids into dysfunction #run
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks #lyft credit cause they offer wheelchair vans #disapointed #getthanked
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model love take with time
4,5,0.0,factsguide: society now #motivation,factsguide society #motivation


In [0]:
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split()) # tokenizing

In [89]:
tokenized_tweet.head(10)

0                                                      [when, father, dysfunctional, selfish, drags, kids, into, dysfunction, #run]
1                                          [thanks, #lyft, credit, cause, they, offer, wheelchair, vans, #disapointed, #getthanked]
2                                                                                                           [bihday, your, majesty]
3                                                                                                  [#model, love, take, with, time]
4                                                                                                [factsguide, society, #motivation]
5                                   [huge, fare, talking, before, they, leave, chaos, disputes, when, they, there, #allshowandnogo]
6                                                                                                        [camping, tomorrow, danny]
7    [next, school, year, year, exams, think, about, that, #school, #exams, 

In [0]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

In [91]:
stemmer.stem("running")

'run'

In [0]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
    
combi['tidy_tweet'] = tokenized_tweet

In [93]:
combi['tidy_tweet'].head(10)

0                                                      when father dysfunct selfish drag kid into dysfunct #run
1                                        thank #lyft credit caus they offer wheelchair van #disapoint #getthank
2                                                                                           bihday your majesti
3                                                                                    #model love take with time
4                                                                                      factsguid societi #motiv
5                                    huge fare talk befor they leav chao disput when they there #allshowandnogo
6                                                                                           camp tomorrow danni
7    next school year year exam think about that #school #exam #hate #imagin #actorslif #revolutionschool #girl
8                                                   love land #allin #cav #champion #cleveland #clevelan

In [0]:
from gensim.models import Word2Vec

In [95]:
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split()) # tokenizing

model_w2v = Word2Vec(tokenized_tweet, size=200,  window=5)

model_w2v.train(tokenized_tweet, total_examples= len(combi['tidy_tweet']), epochs=20)

(5946077, 7536020)

In [96]:
model_w2v.wv.most_similar(positive="dinner")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('lunch', 0.7730880975723267),
 ('breakfast', 0.7233591079711914),
 ('brunch', 0.7215463519096375),
 ('wine', 0.7184600830078125),
 ('#wine', 0.7153099179267883),
 ('pool', 0.6711572408676147),
 ('#brunch', 0.6408213376998901),
 ('chill', 0.6365127563476562),
 ('#steak', 0.6364434957504272),
 ('#soexcit', 0.6343974471092224)]

In [97]:
model_w2v.wv.most_similar(positive="trump")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('hillari', 0.797304630279541),
 ('suppoer', 0.7403860092163086),
 ('campaign', 0.7376409769058228),
 ('obama', 0.725165843963623),
 ('vote', 0.7242649793624878),
 ('voter', 0.7227574586868286),
 ('berni', 0.7211140394210815),
 ('nomine', 0.7074691653251648),
 ('putin', 0.7006750106811523),
 ('donald', 0.6983277797698975)]

In [0]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0:
        vec /= count
    return vec


In [99]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200))

for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(49159, 200)

In [0]:
# from gensim.models import FastText

In [0]:
# print("Training a Gensim FastText model")
# model = FastText(sentences=tokenized_tweet, size = 200, window = 2) # workers = workers, 
# print("Training complete")

In [0]:
# model.most_similar(positive="dinner")

In [0]:
# def word_vector(tokens, size):
#     vec = np.zeros(size).reshape((1, size))
#     count = 0.
#     for word in tokens:
#         try:
#             vec += model[word].reshape((1, size))
#             count += 1.
#         except KeyError: # handling the case where the token is not in vocabulary
                         
#             continue
#     if count != 0:
#         vec /= count
#     return vec

In [0]:
# fasttext_arrays = np.zeros((len(tokenized_tweet), 200))

# for i in range(len(tokenized_tweet)):
#     fasttext_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
    
# fasttext_df = pd.DataFrame(fasttext_arrays)
# fasttext_df.shape

In [100]:
from sklearn.model_selection import train_test_split


train_w2v = wordvec_df.iloc[:31962,:]
test_w2v = wordvec_df.iloc[31962:,:]

_, _, ytrain, yvalid = train_test_split(train_w2v, train['label'],  
                                                          random_state=42, 
                                                          test_size=0.3)



print(train_w2v.shape, train['label'].shape)

xtrain_w2v = train_w2v.iloc[ytrain.index,:]
xvalid_w2v = train_w2v.iloc[yvalid.index,:]

(31962, 200) (31962,)


In [101]:
import tensorflow as tf
import keras.layers as layers
from keras.models import Model
from keras.datasets import imdb

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input,Embedding,Dense,Flatten
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import f1_score

epochs = 25
batch_size = 1024
loss = "binary_crossentropy"
optimizer = "adam"
metrics = ["accuracy"]

from keras import models

callbacks = [EarlyStopping(monitor='val_loss', patience=2),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

# Build neural network
model = models.Sequential()
model.add(Dense(512, activation='relu', input_shape=(200,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss=loss,optimizer=optimizer,metrics= metrics)
model.fit(xtrain_w2v,ytrain,epochs=epochs,batch_size=batch_size,callbacks=callbacks,validation_data=(xvalid_w2v,yvalid))


predictions = model.predict(xvalid_w2v)
predictions = [0 if i<0.5 else 1 for i in predictions]

f1_score(yvalid, predictions)
# print("Accuracy: ",accuracy_score(ytrain,predictions))
# print("Classification Report: ",classification_report(ytrain,predictions))


Train on 22373 samples, validate on 9589 samples
Epoch 1/25
22373/22373 [==============================] - 1s 57us/step - loss: 0.2346 - acc: 0.9080 - val_loss: 0.1694 - val_acc: 0.9402
Epoch 2/25
22373/22373 [==============================] - 1s 41us/step - loss: 0.1521 - acc: 0.9431 - val_loss: 0.1565 - val_acc: 0.9437
Epoch 3/25
22373/22373 [==============================] - 1s 41us/step - loss: 0.1406 - acc: 0.9477 - val_loss: 0.1506 - val_acc: 0.9457
Epoch 4/25
22373/22373 [==============================] - 1s 42us/step - loss: 0.1337 - acc: 0.9510 - val_loss: 0.1467 - val_acc: 0.9464
Epoch 5/25
22373/22373 [==============================] - 1s 43us/step - loss: 0.1276 - acc: 0.9536 - val_loss: 0.1441 - val_acc: 0.9468
Epoch 6/25
22373/22373 [==============================] - 1s 41us/step - loss: 0.1211 - acc: 0.9555 - val_loss: 0.1411 - val_acc: 0.9482
Epoch 7/25
22373/22373 [==============================] - 1s 41us/step - loss: 0.1135 - acc: 0.9585 - val_loss: 0.1388 - val_acc:

0.6119770303527482

In [0]:
test_pred = model.predict(test_w2v)
test_pred = [0 if i<0.5 else 1 for i in test_pred]

test['label'] = test_pred
submission = test[['id','label']]
submission.to_csv('model_result.csv', index=False)


In [102]:
test.head(3)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedication #willpower to find #newmaterialsâ¦
1,31964,@user #white #supremacists want everyone to see the new â #birdsâ #movie â and hereâs why
2,31965,safe ways to heal your #acne!! #altwaystoheal #healthy #healing!!


In [103]:
submission

,id,label
0,31963,0
1,31964,0
2,31965,0
3,31966,0
4,31967,0
...,...,...
17192,49155,1
17193,49156,0
17194,49157,0
17195,49158,0
